- jupyter nbconvert cb_rasa.ipynb --to slides --post serve --ServePostProcessor.port=8889 --SlidesExporter.reveal_scroll=True

# Building chatbot for free with Rasa NLU and Rasa Core

| Pavel Nesterov | Data Scientist |
|---|---|
| <img src="./images/ods.png" />  | <img src="./images/r2.png" />  |
| http://DubaiDataScience.ae/ | https://www.reaktor.com/ |
- http://pavelnesterov.info/
- https://github.com/mephistopheies/dds/tree/master/ibm_290618

# What is a chatbot?
- a chatbot is software you have conversation with

# Why do you want have a chatbot?
- ~~because everyone has it~~
- online mini-version of your business
- extend your marketing
- focus on automation

# Chat API
<img src="./images/api.png" />

# HYPE!!!!!1111
- technology for conversational AI does not exist **yet**

<img src="./images/f64.jpg" />

<img src="./images/cb_types.png" />

# Proprietary chatbot services and frameworks
| Most prominent emaples |  |
|---|---|
| IBM Watson | <img src="./images/watson_conversations_icon.png" width=64/>  |
| LUIS.ai (Microsoft) | <img src="./images/luis.png" width=64/> |
| Microsoft Bot Framework | <img src="./images/mbf.png" width=64/> |
| Dialogflow (former API.ai, Google) | <img src="./images/df.jpg" width=64/> |
| WIT.ai (Facebook) | <img src="./images/witailogo-580x358.png" width=64/> |

# Proprietary chatbot services and frameworks

## Pros
- easy to build, you don't need senior data scientist
- very good in the most popular use cases and happy scenarios
- supports a lot of small talks

## Cons
- closed, no control of what is going on under the hood
- hosted outside of your private network
- not good in edge cases
- do not cover unique use saces (e.g. DOTA player assistance)

# Free open-source frameworks ~~services~~
- DeepPavlov
  - https://github.com/deepmipt/DeepPavlov
  - 2k stars
- Rasa NLU/CORE
  - https://github.com/RasaHQ/rasa_nlu
  - https://github.com/RasaHQ/rasa_core
  - 4k start and 1k stars

<img src="./images/rasa_logo_b.png" />

<img src="./images/rasa-nlu.png" />

<img src="./images/nlu_proc.png" />

```json
{
    "text": "moderately priced restaurant that serves creative food", 
    "intent": "inform", 
    "entities": [
        {
            "start": 41, 
            "end": 49, 
            "value": "creative", 
            "entity": "cuisine"
        }, 
        {
            "start": 0, 
            "end": 10, 
            "value": "moderate", 
            "entity": "price"
        }
    ]
}
```

- https://github.com/RasaHQ/rasa-nlu-trainer
<img src="./images/nlu-trainer.png" />

<img src="./images/rasa-core.png" />

<img src="./images/core_proc.png" />

## story_03812903
* greet
 - utter_ask_howcanhelp
* inform{"location": "paris", "people": "six", "price": "cheap"}
 - utter_on_it
 - utter_ask_cuisine
* inform{"cuisine": "indian"}
 - utter_ask_moreupdates
* inform{"location": "bombay"}
 - utter_ask_moreupdates
* inform{"people": "four"}
 - utter_ask_moreupdates
* inform{"cuisine": "french"}
 - utter_ask_moreupdates
* deny
 - utter_ack_dosearch
 - action_search_restaurants
 - action_suggest
* deny
 - utter_ack_findalternatives
 - action_suggest
* deny
 - utter_ack_findalternatives
 - action_suggest
* affirm
 - utter_ack_makereservation
* thankyou
 - utter_goodbye

<img src="./images/stories.png" />

<img src="./images/rasa_proc.png" />

# Let's build simple bot
- https://github.com/RasaHQ/rasa_core/tree/master/examples/restaurantbot

# NLU Config
```yml
language: "en"

pipeline:
- name: "nlp_spacy"
  model: "en_core_web_md"
  case_sensitive: false
- name: "tokenizer_spacy"
- name: "intent_entity_featurizer_regex"
- name: "intent_featurizer_spacy"
- name: "ner_crf"
  features: [
      ["prefix5", "prefix2", "suffix5", "suffix3", "suffix2", "suffix1"],
      ["suffix2", "suffix1", "pos", "pos2", "bias", "digit", "pattern"],
      ["suffix3", "suffix2", "suffix1", "pos", "pos2", "bias"]
  ]
  BILOU_flag: true
  max_iterations: 500
  L1_c: 1e-1
  L2_c: 1e-3
- name: "ner_synonyms"
- name: "intent_classifier_sklearn"
  C: [1, 2, 5, 10, 20, 100]
  kernels: ["linear"]
```

In [1]:
import os
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer

root_dir = './../../../rasa_core/examples/restaurantbot/'

training_data = load_data(os.path.join(root_dir, 'data/franken_data.json'))

In [2]:
print(training_data.entities)
print(training_data.intents)

{'location', 'price', 'cuisine', 'info', 'people'}
{'thankyou', 'inform', 'greet', 'affirm', 'deny', 'request_info'}


In [7]:
import functools as func
import json
from rasa_nlu.training_data import Message

js = json.loads(training_data.as_json())
examples = js['rasa_nlu_data']['common_examples']
pd.Series(func.reduce(
    lambda a, b: a + b, 
    [[g['entity'] for g in d['entities']] for d in examples if 'entities' in d])).value_counts()

cuisine     573
price       354
location    338
info         16
people       12
dtype: int64

In [8]:
pd.Series([d['intent'] for d in examples]).value_counts()

inform          1014
thankyou         589
affirm           260
deny              85
request_info      16
greet             13
dtype: int64

In [9]:
%%time
trainer = Trainer(config.load(os.path.join(root_dir, 'nlu_model_config.yml')))
trainer.train(training_data)
model_directory = trainer.persist(
    os.path.join(root_dir, 'tmp/'),
    fixed_model_name='current'
)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    5.2s finished


CPU times: user 24.7 s, sys: 1.48 s, total: 26.2 s
Wall time: 22.9 s


In [10]:
from rasa_nlu.model import Interpreter

nlu = Interpreter.load(model_directory)

In [11]:
nlu.parse('hello')

{'intent': {'name': 'greet', 'confidence': 0.9508392002298444},
 'entities': [],
 'intent_ranking': [{'name': 'greet', 'confidence': 0.9508392002298444},
  {'name': 'affirm', 'confidence': 0.023417162583299917},
  {'name': 'thankyou', 'confidence': 0.013442362916337932},
  {'name': 'deny', 'confidence': 0.00568399271695979},
  {'name': 'inform', 'confidence': 0.005056016685371877},
  {'name': 'request_info', 'confidence': 0.0015612648681859868}],
 'text': 'hello'}

In [12]:
nlu.parse('expensive restaurant with indian food in the east part of the town')

{'intent': {'name': 'inform', 'confidence': 0.9757723611117624},
 'entities': [{'start': 0,
   'end': 9,
   'value': 'hi',
   'entity': 'price',
   'confidence': 0.9940886960399758,
   'extractor': 'ner_crf',
   'processors': ['ner_synonyms']},
  {'start': 26,
   'end': 32,
   'value': 'indian',
   'entity': 'cuisine',
   'confidence': 0.9964799053442849,
   'extractor': 'ner_crf'},
  {'start': 45,
   'end': 49,
   'value': 'east',
   'entity': 'location',
   'confidence': 0.9353412350109295,
   'extractor': 'ner_crf'}],
 'intent_ranking': [{'name': 'inform', 'confidence': 0.9757723611117624},
  {'name': 'affirm', 'confidence': 0.013422241845667222},
  {'name': 'request_info', 'confidence': 0.007158413825471512},
  {'name': 'deny', 'confidence': 0.002767319489355853},
  {'name': 'thankyou', 'confidence': 0.000566557072031786},
  {'name': 'greet', 'confidence': 0.0003131066557114439}],
 'text': 'expensive restaurant with indian food in the east part of the town'}

In [13]:
from rasa_core.policies.keras_policy import KerasPolicy

class RestaurantPolicy(KerasPolicy):
    def model_architecture(self, input_shape, output_shape):
        """Build a Keras model and return a compiled model."""
        from keras.layers import LSTM, Activation, Masking, Dense
        from keras.models import Sequential

        from keras.models import Sequential
        from keras.layers import \
            Masking, LSTM, Dense, TimeDistributed, Activation


        model = Sequential()

        # the shape of the y vector of the labels,
        # determines which output from rnn will be used
        # to calculate the loss
        if len(output_shape) == 1:
            # y is (num examples, num features) so
            # only the last output from the rnn is used to
            # calculate the loss
            model.add(Masking(mask_value=-1, input_shape=input_shape))
            model.add(LSTM(self.rnn_size))
            model.add(Dense(input_dim=self.rnn_size, units=output_shape[-1]))
        elif len(output_shape) == 2:
            # y is (num examples, max_dialogue_len, num features) so
            # all the outputs from the rnn are used to
            # calculate the loss, therefore a sequence is returned and
            # time distributed layer is used

            # the first value in input_shape is max dialogue_len,
            # it is set to None, to allow dynamic_rnn creation
            # during prediction
            model.add(Masking(mask_value=-1,
                              input_shape=(None, input_shape[1])))
            model.add(LSTM(self.rnn_size, return_sequences=True))
            model.add(TimeDistributed(Dense(units=output_shape[-1])))
        else:
            raise ValueError("Cannot construct the model because"
                             "length of output_shape = {} "
                             "should be 1 or 2."
                             "".format(len(output_shape)))

        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        
        return model

In [14]:
%%time
from rasa_core.agent import Agent
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.policies.keras_policy import KerasPolicy

domain_file = os.path.join(root_dir, 'restaurant_domain_fixed.yml')
training_data_file = os.path.join(root_dir, 'data/babi_stories_fixed.md')
    
agent = Agent(
    domain_file,
    policies=[
        MemoizationPolicy(max_history=3),
        KerasPolicy()
    ])

training_data = agent.load_data(training_data_file)

Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|██████████| 1000/1000 [00:03<00:00, 304.42it/s, # trackers=1]


CPU times: user 12.3 s, sys: 2.01 s, total: 14.3 s
Wall time: 13.4 s


In [15]:
%%time
agent.train(
    training_data,
    epochs=100,
    batch_size=100,
    validation_split=0.2)

agent_directory = agent.persist(os.path.join(root_dir, 'tmp/core/'))

Processed trackers: 100%|██████████| 2911/2911 [03:26<00:00, 14.07it/s, # actions=236]
Processed actions: 236it [00:00, 676.80it/s, # examples=227]
Processed trackers: 100%|██████████| 2911/2911 [03:26<00:00, 14.10it/s, # actions=511]


Train on 408 samples, validate on 103 samples
Epoch 1/100
408/408 [==============================] - 1s 2ms/step - loss: 2.9575 - acc: 0.0245 - val_loss: 2.8598 - val_acc: 0.0583
Epoch 2/100
408/408 [==============================] - 0s 80us/step - loss: 2.8318 - acc: 0.0392 - val_loss: 2.7361 - val_acc: 0.3398
Epoch 3/100
408/408 [==============================] - 0s 81us/step - loss: 2.7125 - acc: 0.3799 - val_loss: 2.6110 - val_acc: 0.4272
Epoch 4/100
408/408 [==============================] - 0s 79us/step - loss: 2.5902 - acc: 0.4338 - val_loss: 2.4791 - val_acc: 0.4272
Epoch 5/100
408/408 [==============================] - 0s 80us/step - loss: 2.4606 - acc: 0.4412 - val_loss: 2.3399 - val_acc: 0.4272
Epoch 6/100
408/408 [==============================] - 0s 80us/step - loss: 2.3249 - acc: 0.4436 - val_loss: 2.1934 - val_acc: 0.4272
Epoch 7/100
408/408 [==============================] - 0s 80us/step - loss: 2.1788 - acc: 0.4436 - val_loss: 2.0471 - val_acc: 0.4272
Epoch 8/100
408/4

408/408 [==============================] - 0s 79us/step - loss: 0.5987 - acc: 0.8358 - val_loss: 0.4919 - val_acc: 0.8641
Epoch 62/100
408/408 [==============================] - 0s 79us/step - loss: 0.5836 - acc: 0.8358 - val_loss: 0.4800 - val_acc: 0.8641
Epoch 63/100
408/408 [==============================] - 0s 81us/step - loss: 0.5709 - acc: 0.8431 - val_loss: 0.4707 - val_acc: 0.8641
Epoch 64/100
408/408 [==============================] - 0s 80us/step - loss: 0.5577 - acc: 0.8407 - val_loss: 0.4623 - val_acc: 0.8641
Epoch 65/100
408/408 [==============================] - 0s 79us/step - loss: 0.5440 - acc: 0.8407 - val_loss: 0.4531 - val_acc: 0.8641
Epoch 66/100
408/408 [==============================] - 0s 81us/step - loss: 0.5328 - acc: 0.8480 - val_loss: 0.4417 - val_acc: 0.8544
Epoch 67/100
408/408 [==============================] - 0s 81us/step - loss: 0.5229 - acc: 0.8456 - val_loss: 0.4300 - val_acc: 0.8544
Epoch 68/100
408/408 [==============================] - 0s 80us/step

In [16]:
from rasa_core.interpreter import RasaNLUInterpreter

agent = Agent.load(
    os.path.join(root_dir, 'tmp/core/'), 
    interpreter=RasaNLUInterpreter(model_directory))

/usr/local/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)


In [134]:
import uuid

sender_id = str(uuid.uuid4())
print(sender_id)

241df086-0dd2-49ac-94df-09c142762ed1


In [135]:
msg = 'hello'

answer = agent.handle_message(msg, sender_id=sender_id)
tracker = agent.tracker_store.get_or_create_tracker(sender_id)
state = tracker.current_state()
for a in answer:
    print(a['text'])

how can I help you?


In [136]:
state

{'sender_id': '241df086-0dd2-49ac-94df-09c142762ed1',
 'slots': {'cuisine': None,
  'info': None,
  'location': None,
  'matches': None,
  'people': None,
  'price': None},
 'latest_message': {'intent': {'name': 'greet',
   'confidence': 0.9508392002298444},
  'entities': [],
  'intent_ranking': [{'name': 'greet', 'confidence': 0.9508392002298444},
   {'name': 'affirm', 'confidence': 0.023417162583299917},
   {'name': 'thankyou', 'confidence': 0.013442362916337932},
   {'name': 'deny', 'confidence': 0.00568399271695979},
   {'name': 'inform', 'confidence': 0.005056016685371877},
   {'name': 'request_info', 'confidence': 0.0015612648681859868}],
  'text': 'hello'},
 'latest_event_time': 1535745355.450332,
 'paused': False,
 'events': None}

In [137]:
msg = 'moderately priced restaurant in the south part of town'

answer = agent.handle_message(msg, sender_id=sender_id)
tracker = agent.tracker_store.get_or_create_tracker(sender_id)
state = tracker.current_state()
for a in answer:
    print(a['text'])

I'm on it
what kind of cuisine would you like?


In [138]:
state

{'sender_id': '241df086-0dd2-49ac-94df-09c142762ed1',
 'slots': {'cuisine': None,
  'info': None,
  'location': 'south',
  'matches': None,
  'people': None,
  'price': 'moderate'},
 'latest_message': {'intent': {'name': 'inform',
   'confidence': 0.9746576553738102},
  'entities': [{'start': 0,
    'end': 10,
    'value': 'moderate',
    'entity': 'price',
    'confidence': 0.9965307924983892,
    'extractor': 'ner_crf',
    'processors': ['ner_synonyms']},
   {'start': 36,
    'end': 41,
    'value': 'south',
    'entity': 'location',
    'confidence': 0.9490795643500979,
    'extractor': 'ner_crf'}],
  'intent_ranking': [{'name': 'inform', 'confidence': 0.9746576553738102},
   {'name': 'affirm', 'confidence': 0.021239525703462523},
   {'name': 'deny', 'confidence': 0.0020033534350543786},
   {'name': 'request_info', 'confidence': 0.0013917180993503549},
   {'name': 'thankyou', 'confidence': 0.0005153209797594731},
   {'name': 'greet', 'confidence': 0.00019242640856293166}],
  'text'

In [139]:
msg = 'what about indian style?'

answer = agent.handle_message(msg, sender_id=sender_id)
tracker = agent.tracker_store.get_or_create_tracker(sender_id)
state = tracker.current_state()
for a in answer:
    print(a['text'])

for how many people?


In [140]:
state

{'sender_id': '241df086-0dd2-49ac-94df-09c142762ed1',
 'slots': {'cuisine': 'indian',
  'info': None,
  'location': 'south',
  'matches': None,
  'people': None,
  'price': 'moderate'},
 'latest_message': {'intent': {'name': 'inform',
   'confidence': 0.9658233952171321},
  'entities': [{'start': 11,
    'end': 17,
    'value': 'indian',
    'entity': 'cuisine',
    'confidence': 0.9837364633497523,
    'extractor': 'ner_crf'}],
  'intent_ranking': [{'name': 'inform', 'confidence': 0.9658233952171321},
   {'name': 'affirm', 'confidence': 0.02915325031086175},
   {'name': 'thankyou', 'confidence': 0.0025338624138773287},
   {'name': 'deny', 'confidence': 0.0011459007617097244},
   {'name': 'greet', 'confidence': 0.000986190201556008},
   {'name': 'request_info', 'confidence': 0.00035740109486263877}],
  'text': 'what about indian style?'},
 'latest_event_time': 1535745357.827314,
 'paused': False,
 'events': None}

In [141]:
msg = 'six people'

answer = agent.handle_message(msg, sender_id=sender_id)
tracker = agent.tracker_store.get_or_create_tracker(sender_id)
state = tracker.current_state()
for a in answer:
    print(a['text'])

if you'd like to modify anything else, please tell me what. This is what I currently have: south (price: moderate, cuisine: indian) for 6 people.


In [142]:
state

{'sender_id': '241df086-0dd2-49ac-94df-09c142762ed1',
 'slots': {'cuisine': 'indian',
  'info': None,
  'location': 'south',
  'matches': None,
  'people': '6',
  'price': 'moderate'},
 'latest_message': {'intent': {'name': 'inform',
   'confidence': 0.732600112586828},
  'entities': [{'start': 0,
    'end': 3,
    'value': '6',
    'entity': 'people',
    'confidence': 0.6296027592657509,
    'extractor': 'ner_crf',
    'processors': ['ner_synonyms']}],
  'intent_ranking': [{'name': 'inform', 'confidence': 0.732600112586828},
   {'name': 'request_info', 'confidence': 0.1366190653850414},
   {'name': 'thankyou', 'confidence': 0.07173080451246215},
   {'name': 'deny', 'confidence': 0.02771122608867573},
   {'name': 'greet', 'confidence': 0.017306430013090193},
   {'name': 'affirm', 'confidence': 0.014032361413902062}],
  'text': 'six people'},
 'latest_event_time': 1535745359.5551639,
 'paused': False,
 'events': None}

In [143]:
msg = 'no'

answer = agent.handle_message(msg, sender_id=sender_id)
tracker = agent.tracker_store.get_or_create_tracker(sender_id)
state = tracker.current_state()
for a in answer:
    print(a['text'])

ok let me see what I can find
MOCK action_search_restaurants
MOCK action_suggest


In [144]:
state

{'sender_id': '241df086-0dd2-49ac-94df-09c142762ed1',
 'slots': {'cuisine': 'indian',
  'info': None,
  'location': 'south',
  'matches': None,
  'people': '6',
  'price': 'moderate'},
 'latest_message': {'intent': {'name': 'deny',
   'confidence': 0.9967124904217057},
  'entities': [],
  'intent_ranking': [{'name': 'deny', 'confidence': 0.9967124904217057},
   {'name': 'greet', 'confidence': 0.002621788099360835},
   {'name': 'inform', 'confidence': 0.000339662415716998},
   {'name': 'request_info', 'confidence': 0.00017697615665696764},
   {'name': 'thankyou', 'confidence': 0.00010985058434942631},
   {'name': 'affirm', 'confidence': 3.923232220997178e-05}],
  'text': 'no'},
 'latest_event_time': 1535745361.13536,
 'paused': False,
 'events': None}

In [145]:
msg = 'yes'

answer = agent.handle_message(msg, sender_id=sender_id)
tracker = agent.tracker_store.get_or_create_tracker(sender_id)
state = tracker.current_state()
for a in answer:
    print(a['text'])

ok making a reservation for restaurant (price=moderate cuisine=indian) in location=south for count=6? 


<img src="./images/end.png" />